# LSTM Implementation

In [43]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import TextClassificationPipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.optim import SGD
from torch.optim.lr_scheduler import MultiStepLR
from tqdm import tqdm

## Load Data

In [44]:
# Import train and test data
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Separate train labels and text
train_labels = train['category']
train_text = train['text']
train_labels_list = train_labels.tolist()

# Separate test labels and text
test_labels = test['category']
test_text = test['text']
test_labels_list = test_labels.tolist()

### Randomly select 10% of the dataset to be noised

In [45]:
# Set the random seed for reproducibility
np.random.seed(42)

# Calculate the number of samples for 10% of the dataset
sample_size = int(len(train) * 0.1)

# Conduct sampling
sampled_data = train.sample(n=sample_size, replace=False)

sampled_data

,text,category
6883,Is it possible for me to change my PIN number?,change_pin
5836,I'm not sure why my card didn't work,declined_card_payment
8601,I don't think my top up worked,top_up_failed
2545,Can you explain why my payment was charged a fee?,card_payment_fee_charged
8697,How long does a transfer from a UK account tak...,balance_not_updated_after_bank_transfer
...,...,...
7089,I have been trying to exchange this for crypto...,beneficiary_not_allowed
4238,I want to transfer money using my credit card.,topping_up_by_card
4901,I can't remember what my password is,passcode_forgotten
3458,My cheque is taking a while,balance_not_updated_after_cheque_or_cash_deposit


### Shuffle 10% of Labels in Dn to Create Noise

In [46]:
# Randomly sample 10% of the rows from the Dn
sampled_rows = sampled_data.sample(frac=0.1, random_state=42)

# Add new column 'k' and initialize with 0
sampled_data['k'] = 0

# Store the original labels ('l') before shuffling the 'category' column
sampled_data['l'] = sampled_data['category']  # Initialize 'l' column with original category values
sampled_data.loc[sampled_rows.index, 'l'] = sampled_data.loc[sampled_rows.index, 'category']

# Convert the selected rows' 'category' column to a numpy array
sampled_category_values = np.array(sampled_rows['category'])

# Shuffle the array
np.random.shuffle(sampled_category_values)

# Update the 'category' column with the shuffled values in the original DataFrame
sampled_data.loc[sampled_rows.index, 'category'] = sampled_category_values

# Update 'k' to 1 for the randomly selected 10% of rows
sampled_data.loc[sampled_rows.index, 'k'] = 1

# Update 'l\'' for the shuffled rows
sampled_data.loc[sampled_rows.index, 'l\''] = sampled_category_values

# Display the DataFrame to verify the changes
sampled_data.head(25)


,text,category,k,l,l'
6883,Is it possible for me to change my PIN number?,change_pin,0,change_pin,NaN
5836,I'm not sure why my card didn't work,declined_card_payment,0,declined_card_payment,NaN
8601,I don't think my top up worked,top_up_failed,0,top_up_failed,NaN
2545,Can you explain why my payment was charged a fee?,card_payment_fee_charged,0,card_payment_fee_charged,NaN
8697,How long does a transfer from a UK account tak...,balance_not_updated_after_bank_transfer,0,balance_not_updated_after_bank_transfer,NaN
5573,Why am I getting declines when trying to make ...,declined_transfer,0,declined_transfer,NaN
576,What is the $1 transaction on my account?,extra_charge_on_statement,0,extra_charge_on_statement,NaN
6832,It looks like my card payment was sent back.,reverted_card_payment?,0,reverted_card_payment?,NaN
7111,Why am I unable to transfer money when I was a...,beneficiary_not_allowed,0,beneficiary_not_allowed,NaN
439,What if there is an error on the exchange rate?,card_payment_wrong_exchange_rate,0,card_payment_wrong_exchange_rate,NaN


## Step 1: Define and Train Deep Model

In [47]:
device_name = 'cuda'
model_name = 'bert-base-uncased'

# Input Text
input_text = np.array(sampled_data['text'])
labels = np.array(sampled_data['category'])

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_name)

# Define labels
unique_labels = set(label for label in labels)
label2id = {label: id for id, label in enumerate(unique_labels)}
id2label = {id: label for label, id in label2id.items()}

# Tokenize input text
input_ids = tokenizer.batch_encode_plus(input_text, padding=True, truncation=True, return_tensors='pt')['input_ids']
attention_mask = tokenizer.batch_encode_plus(input_text, padding=True, truncation=True, return_tensors='pt')['attention_mask']

# Tokenize input labels
labels_encoded = [label2id[label] for label in labels]
labels_tensor = torch.tensor(labels_encoded)


# labels_tensor with DataLoader
dataset = torch.utils.data.TensorDataset(input_ids, attention_mask, labels_tensor)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

In [50]:
from safetensors import safe_open

tensors = {}
with safe_open("/Users/wally/Documents/Zeta_Test/AmEx-Project/model.safetensors", framework="pt", device=0) as f:
    for k in f.keys():
        tensors[k] = f.get_tensor(k) # loads the full tensor given a key

AssertionError: Torch not compiled with CUDA enabled

In [14]:
# Load pre-trained model
model = BertForSequenceClassification.from_pretrained('/Users/wally/Documents/Zeta_Test/AmEx-Project/model.safetensors')

# Define optimizer with Adam
optimizer = AdamW(model.parameters(), lr=1e-6, weight_decay=1e-4)

# Define training parameters
num_epochs = 115
batch_size = 16

# List to store probabilities per epoch
probabilities_per_epoch = []

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []  # List to store probabilities for this epoch

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}"):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device_name), attention_mask.to(device_name), labels.to(device_name)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        logits = outputs.logits
        probs = torch.softmax(logits, dim=1)  # Calculate probabilities from logits
        preds = torch.argmax(logits, dim=1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(labels.cpu().tolist())
        all_probs.extend(probs.cpu().tolist())  # Append probabilities to the list

        loss.backward()
        optimizer.step()

    # Calculate F1 score
    f1 = f1_score(all_labels, all_preds, average='macro')

    # Store probabilities for this epoch
    probabilities_per_epoch.append(all_probs)

    print(f"Epoch {epoch+1}, Loss: {total_loss}, F1 Score: {f1}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

AssertionError: Torch not compiled with CUDA enabled